In [120]:
import json

import pandas as pd
import numpy as np
import networkx as nx

import torch
from torch_geometric.nn import GATConv, GATv2Conv

In [15]:
with open("data/preprocessed/price.json") as json_data:
    price = json.load(json_data)

In [17]:
price.keys()

dict_keys(['5506', '6695', '6976', '9273', '9765', '9824', '9855', '9857', '9858', '9860', '9870', '9903', '10758', '10807', '10809', '10810', '10811', '12441', '13059', '13589', '13592', '18711', '20000', '21337', '21891', '22208', '22212', '22354', '23619', '23668', '24240', '25142', '25418', '27141', '27611', '28150', '29155', '29765', '31796', '36885', '38579', '44205', '45987', '47521', '47722', '47725', '48612', '50032', '52423', '54215', '54487', '54944', '56927', '56937', '57800', '59776', '66288', '67946', '68902', '69369', '72811', '76073', '77681', '77691', '82637', '91588', '93505', '95453', '115936', '118905', '130552', '163941', '164418', '169430', '169938', '176006', '179244', '179245', '180914', '182049', '182613', '189965', '195515', '196806', '197146', '197727', '197972', '200265', '205894', '210097', '217498', '218259', '220676', '225224', '225834', '225979', '228883', '232990', '238846', '250345', '257588', '276450', '295285', '311240', '315982', '321328', '322593',

In [51]:
listings = pd.read_csv("data/preprocessed/listings.csv", index_col=0)
index2drop = listings[listings['id'] == 12898806].index.item()
listings = listings.drop(index2drop)

node_order = listings['id'].to_list()

node_order[:3]

[3075044, 6976, 1436513]

In [56]:
adjacency = np.load("data/adjacency/sigma1.npy")

print(adjacency.shape)

adjacency = np.delete(adjacency, index2drop, 0)
adjacency = np.delete(adjacency, index2drop, 1)

adjacency.shape

(2629, 2629)


(2628, 2628)

In [171]:
class GATv3Conv(torch.nn.Module):
    def __init__(self, in_channels, out_channels, heads=1) -> None:
        super().__init__()
        self.beta = torch.nn.Parameter(torch.tensor(0.5))
        self.conv = GATv2Conv(in_channels, out_channels, heads, add_self_loops=False)

    def forward(self, x, edge_index, edge_weights):
        H, C = self.conv.heads, self.conv.out_channels

        if isinstance(x, torch.Tensor):
            assert x.dim() == 2
            x_l = self.conv.lin_l(x).view(-1, H, C)
            if self.conv.share_weights:
                x_r = x_l
            else:
                x_r = self.conv.lin_r(x).view(-1, H, C)
        else:
            raise TypeError("x must be a Tensor")

        assert x_l is not None
        assert x_r is not None

        # edge_updater_type: (x: PairTensor, edge_attr: OptTensor)
        alpha = self.conv.edge_updater(edge_index, x=(x_l, x_r), edge_attr=None)

        alpha = (1-self.beta) * alpha + self.beta * edge_weights

        # propagate_type: (x: PairTensor, alpha: Tensor)
        out = self.conv.propagate(edge_index, x=(x_l, x_r), alpha=alpha)

        if self.conv.concat:
            out = out.view(-1, self.conv.heads * self.conv.out_channels)
        else:
            out = out.mean(dim=1)

        if self.conv.bias is not None:
            out = out + self.conv.bias

        return out
    

class GATNN(torch.nn.Module):
    def __init__(self, hidden_size, in_dim, out_dim) -> None:
        super().__init__()
        self.conv1 = GATv3Conv(in_dim, hidden_size)
        self.conv2 = GATv3Conv(hidden_size, out_dim)

    def forward(self, x, edge_index, edge_weight):
        x = self.conv1(x, edge_index, edge_weight)
        x = self.conv2(x, edge_index, edge_weight)
        return x

In [219]:
nrFeatures = 1

800/(nrFeatures*4)

200.0

In [172]:
gat_net = GATNN(16, 1, 1)
gat_net

GATNN(
  (conv1): GATv3Conv(
    (conv): GATv2Conv(1, 16, heads=1)
  )
  (conv2): GATv3Conv(
    (conv): GATv2Conv(16, 1, heads=1)
  )
)

In [201]:
x = torch.tensor([[100], 
                  [1],
                  [2],
                  [3]]).float()

edge_index = torch.tensor([[0, 1, 1, 2, 3, 2], 
                           [1, 0, 2, 1, 2, 3]])

edge_weights = torch.tensor([[0.3],
                             [0.3],
                             [0.9],
                             [0.9],
                             [0.1],
                             [0.1]])

gat_net(x, edge_index, edge_weights)

tensor([[-5.4292],
        [-0.9369],
        [-4.6689],
        [-0.5501]], grad_fn=<AddBackward0>)